In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from tkinter import ttk
from PIL import Image, ImageTk
import webcolors

In [2]:
class ColorDetectorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Color Detection")
        self.root.geometry("900x700")
        self.root.resizable(False, False)

        # Top frame for buttons
        self.top_frame = tk.Frame(self.root, bg="#4CAF50")
        self.top_frame.pack(side=tk.TOP, fill=tk.X)

        self.upload_btn = tk.Button(self.top_frame, text="Upload Image", font=("Arial", 12), bg="#FFFFFF",
                                    command=self.upload_image)
        self.upload_btn.pack(side=tk.LEFT, padx=10, pady=10)

        self.clear_btn = tk.Button(self.top_frame, text="Clear", font=("Arial", 12), bg="#FFFFFF",
                                   command=self.clear_canvas)
        self.clear_btn.pack(side=tk.LEFT, padx=10, pady=10)

        # Canvas for displaying image
        self.canvas = tk.Canvas(self.root, bg="white", width=600, height=500)
        self.canvas.pack(pady=10)

        # Color Information Frame
        self.info_frame = tk.Frame(self.root, bg="#F1F1F1", relief=tk.RIDGE, borderwidth=2)
        self.info_frame.pack(fill=tk.X, pady=10)

        self.color_label = tk.Label(self.info_frame, text="Click on the image to detect color", font=("Arial", 14), bg="#F1F1F1")
        self.color_label.pack(side=tk.LEFT, padx=10, pady=10)

        # Color History Table
        self.history_label = tk.Label(self.root, text="Detected Color History", font=("Arial", 14, "bold"))
        self.history_label.pack(pady=5)

        self.history_table = ttk.Treeview(self.root, columns=("RGB", "Hex"), show="headings", height=10)
        self.history_table.heading("RGB", text="RGB")
        self.history_table.heading("Hex", text="Hex Code")
        self.history_table.column("RGB", width=200, anchor="center")
        self.history_table.column("Hex", width=200, anchor="center")
        self.history_table.pack(pady=5)

        self.image = None
        self.tk_image = None

    def upload_image(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.png;*.jpg;*.jpeg;*.bmp")])
        if not file_path:
            return
        
        try:
            self.image = Image.open(file_path)
            self.image.thumbnail((600, 500))  # Resize for fitting the canvas while maintaining aspect ratio
            self.tk_image = ImageTk.PhotoImage(self.image)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=self.tk_image)
            self.canvas.bind("<Button-1>", self.detect_color)
            self.color_label.config(text="Click on the image to detect color", bg="#F1F1F1")
        except Exception as e:
            messagebox.showerror("Error", f"Failed to load image: {e}")

    def detect_color(self, event):
        if self.image:
            # Calculate click coordinates on the original image
            canvas_width = self.canvas.winfo_width()
            canvas_height = self.canvas.winfo_height()
            image_width, image_height = self.image.size

            scale_x = image_width / canvas_width
            scale_y = image_height / canvas_height

            x = int(event.x * scale_x)
            y = int(event.y * scale_y)

            try:
                # Get the pixel color
                pixel_color = self.image.getpixel((x, y))
            
                # Handle RGBA images by using only the first three values (R, G, B)
                if len(pixel_color) == 4:  # If RGBA, discard the alpha channel
                    rgb_color = pixel_color[:3]
                else:
                    rgb_color = pixel_color

                # Convert RGB to Hex
                hex_color = "#%02x%02x%02x" % rgb_color
                
                # Update the label and history table
                self.color_label.config(text=f"RGB: {rgb_color}  Hex: {hex_color}", bg=hex_color)
                self.history_table.insert("", "end", values=(rgb_color, hex_color))
            except IndexError:
                messagebox.showwarning("Warning", "Clicked outside the image area!")

    def clear_canvas(self):
        self.canvas.delete("all")
        self.color_label.config(text="Click on the image to detect color", bg="#F1F1F1")
        self.history_table.delete(*self.history_table.get_children())
        self.image = None
        self.tk_image = None

In [3]:
def run_app():
    root = tk.Tk()
    app = ColorDetectorApp(root)
    root.mainloop()

run_app()